# Food Delivery Data Analysis

This notebook analyzes food delivery data from three sources: CSV (orders), JSON (users), and SQL (restaurants).
It merges these datasets and answers key business questions.

In [ ]:
import pandas as pd
import re
import ast

# Load CSV Data (Orders)
orders_df = pd.read_csv('orders.csv')
print("Orders Loaded:", orders_df.shape)

# Load JSON Data (Users)
users_df = pd.read_json('users.json')
print("Users Loaded:", users_df.shape)

# Load SQL Data (Restaurants)
# Parsing INSERT statements from the SQL file
restaurants_data = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        if line.strip().startswith('INSERT INTO restaurants VALUES'):
            match = re.search(r'\((.*)\);', line)
            if match:
                values_str = match.group(1)
                try:
                    # Safely evaluate the tuple string
                    values = ast.literal_eval(f"({values_str})")
                    restaurants_data.append(values)
                except Exception as e:
                    print(f"Error parsing line: {e}")

restaurants_df = pd.DataFrame(restaurants_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
print("Restaurants Loaded:", restaurants_df.shape)

## Data Merging
We perform a Left Join to combine orders with user and restaurant details.

In [ ]:
# Merge Orders with Users
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# Merge with Restaurants
# We use suffixes to handle the duplicate 'restaurant_name' column found in orders.csv
final_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left', suffixes=('', '_master'))

# Convert order_date to datetime
final_df['order_date'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')

print("Final Dataset Shape:", final_df.shape)
final_df.head()

## Analysis
Answering key questions about revenue, user behavior, and restaurant performance.

In [ ]:
# 1. Top Revenue Cities
city_revenue = final_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
print("Top Revenue Cities:\n", city_revenue)

# 2. Top Cuisines by Revenue
cuisine_revenue = final_df.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False)
print("\nTop Cuisines:\n", cuisine_revenue)

# 3. Average Order Value by Membership
membership_aov = final_df.groupby('membership')['total_amount'].mean()
print("\nAverage Order Value by Membership:\n", membership_aov)

In [ ]:
# Save the final dataset
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Dataset saved successfully!")